# Bosonic.py

In [23]:
import numpy as np
from scipy.special import binom 
from scipy.sparse import dok_matrix
from joblib import Memory
import random

location = './cachedir'
memory = Memory(location, verbose=0)

In [49]:
class fixed_basis:

    # Convierte a un enterno n a su escritura en base b
    def int_to_tuple(self, n, b, digits = None):
        r = []
        re = n 
        while re != 0:
            r.append(re%b)
            re = re // b
        if digits is not None:
            if len(r)<digits:
                for i in range(0,digits-len(r)):
                    r.append(0)
        r.reverse()
        return r
    
    # Revierte la transformacion anterior
    def tuple_to_int(self, t):
        b = self.d-1
        l = len(t)
        s = [t[k]*b**(l-k-1) for k in range(0,l)]
        return sum(s)

    # Convierte el vector en su representacion
    def vect_to_repr(self, vect):
        for i, k in enumerate(vect):
            if k == 1. or k == 1:
                break
        return self.base[i,:]
            
    def rep_to_vect(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return self.canonicals[:,i]
        else:
            None
    
    def rep_to_index(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return i
        else:
            None
        
    # Crea base de M particulas en D estados (repr y base canonica)
    def create_basis(self, m, d):
        length = int(binom(d+m-1,d-1))
        base = []
        index = 0
        for x in range(0,(d+1)**max(d,m)):
            x = self.int_to_tuple(x,m+1,d)
            if sum(x) == m and len(x) == d:
                base.append(x)
                index += 1
        base = np.array(base)
        # Asignamos a cada uno de ellos un canónico
        x = [1.0 for j in range(0,length)]
        canonicals = np.diag(x)
        return base, canonicals
    
    def __init__(self, m, d):
        self.m = m
        self.d = d
        self.size = int(binom(d+m-1,d-1))
        (self.base, self.canonicals) = self.create_basis(m, d)

# Matrices de aniquilación y creación endomórficas. Estan fuera de la clase para poder ser cacheadas        
@memory.cache    
def bdb(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k, v in enumerate(basis.base):
        if v[j] != 0:
            dest = list(v.copy())
            dest[j] -= 1
            dest[i] += 1
            tar = basis.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i]+1)*np.sqrt(v[j])
    return mat

@memory.cache    
def bbd(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k, v in enumerate(basis.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1
            dest[j] += 1
            tar = basis.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[j]+1)*np.sqrt(v[i])
    return mat

# Matrices de aniquilación y creación.Toman la base de origen y destino (basis_o, basis_d) resp
@memory.cache   
def b(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1   
            tar = basis_d.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i])
    return mat

@memory.cache   
def bd(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        dest = list(v.copy())
        dest[i] += 1   
        tar = basis_d.rep_to_index(dest)
        mat[tar, k] = np.sqrt(v[i]+1)
    return mat

# Devuelve la matriz gamma asociada a la descomposición (M,N-M) del vector
def gamma(basis, m, vect):
    d = basis.d
    m_basis = fixed_basis(m, d)
    nm_basis = fixed_basis(basis.m-m,d)
    mat = dok_matrix((m_basis.size, nm_basis.size), dtype=np.float32)
    for i, v in enumerate(m_basis.base):
        for j, w in enumerate(nm_basis.base):
            targ = v+w
            index = basis.rep_to_index(targ)
            if index == None:
                continue
            coef = vect[index]
            if coef != 0:      
                aux = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
                coef = coef * aux(v) * aux(w)
                print(v,w,coef)
            mat[i,j] = coef
    return mat

# Devuelve la matriz rho M asociada al vector
def rho_m(basis, m, vect):
    gamma = gamma(basis, m, vect)
    return np.dot(gamma,np.transpose(gamma))

In [24]:
m = 4
d = 4
a = fixed_basis(m, d)
print(a.base)
a.rep_to_index(np.array([0, 1, 1]))

[[0 0 0 4]
 [0 0 1 3]
 [0 0 2 2]
 [0 0 3 1]
 [0 0 4 0]
 [0 1 0 3]
 [0 1 1 2]
 [0 1 2 1]
 [0 1 3 0]
 [0 2 0 2]
 [0 2 1 1]
 [0 2 2 0]
 [0 3 0 1]
 [0 3 1 0]
 [0 4 0 0]
 [1 0 0 3]
 [1 0 1 2]
 [1 0 2 1]
 [1 0 3 0]
 [1 1 0 2]
 [1 1 1 1]
 [1 1 2 0]
 [1 2 0 1]
 [1 2 1 0]
 [1 3 0 0]
 [2 0 0 2]
 [2 0 1 1]
 [2 0 2 0]
 [2 1 0 1]
 [2 1 1 0]
 [2 2 0 0]
 [3 0 0 1]
 [3 0 1 0]
 [3 1 0 0]
 [4 0 0 0]]


In [5]:
# Prueba de las matrices gamma (TODO normalizacion)
v = a.rep_to_vect([0, 1, 1, 2])
print(v)
gamma(a, 2, v)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 2] [0 1 1 0] 1.4142135623730951
[0 0 1 1] [0 1 0 1] 1.0
[0 1 0 1] [0 0 1 1] 1.0
[0 1 1 0] [0 0 0 2] 1.4142135623730951


<10x10 sparse matrix of type '<class 'numpy.float32'>'
	with 4 stored elements in Dictionary Of Keys format>

In [ ]:
# Prueba de funcionamiento de las matrices bdb y bbd
v = [1, 1, 0]
vect = a.rep_to_vect(v)
res = bbd(a,0,1) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(a.vect_to_repr(res))
else:
    res = 0
    print(0)


In [55]:
# Prueba de funcionamiento de las matrices b y bd

l = fixed_basis(3, 3)
m = fixed_basis(2, 3)

vect = l.rep_to_vect([0,2,1])
res = b(l,m,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    #print(m.vect_to_repr(res))
    
vect = m.rep_to_vect([1,0,1])
res = bd(m,l,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(l.vect_to_repr(res))

[1 0 2]


## Pairing

Construimos el espacio para n particulas en d estados. El hamiltoniano de pairing posee energias equiespaciadas con degeneración doble (asociada a cada estado y su inverso temporal), así como un término de interacción. Por tal motivo, crearemos un espacio con d = 2d.

In [19]:
m = 4
d = 2
base = fixed_basis(m, 2*d)
# Parametros hamiltoniano
e = 1
eps = 0.01

# Construccion de H
e0 = np.zeros(2*d)
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

In [20]:
np.zeros(base.size)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [21]:
e0

array([0.00718019, 0.00718019, 1.00922468, 1.00922468])